## Installation of Libraries Required

In [5]:
pip install opencv-python numpy tensorflow

Note: you may need to restart the kernel to use updated packages.


## Importing Libraries & Defining the Image Capturing Device

In [6]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import *
from keras.layers import *
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

print(tf.__version__)


2.15.0


In [38]:
cap=cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

False

# Problem Statement

#### This project intends to provide an alternative to the fingerprint scanners at our boarding school using facial recognition. The project only intends to cover the backend, and there will be no physical prototype. Hardware used for demonstration will be web camera from computer.

# Data Collection

The way we intend to obtain our training data is to get a few images of the students and perform one-shot learning with them, such that we do not have to waste the students' time with gathering large quantities of pictures for each of them.

Firstly, we will be using opencv to make use of the webcam as the input device, since we are not building a physical prototype. We also use a haar cascade classifier in order to detect whether there are faces in the webcame image.

In [7]:
# Use the first video capturing device on your device
video=cv2.VideoCapture(0)

# For this, copy the absolute path and replace it with your own
facedetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Check if cascade classifier loaded (whether path to cascade classifier is valid)
if facedetect.empty():
    print("Error: Cascade Classifier not loaded")
else:
    print("Cascade Classifier loaded successfully")

Cascade Classifier loaded successfully


We then run this chunk where the user will be asked to input their name (not case-sensitive) and the webcam will start capturing images of the person if there is a face in the webcam. The images are then cropped such that only the face is visible in the image. 15 pictures total are taken of the student and saved to a folder which will be used for training and testing the model.

In [8]:
count=0
test = False
nameID=str(input("Enter Your Name: ")).lower()

path='images/'+nameID
isExist = os.path.exists(path)

# If name is already taken, ask for another one
if isExist:
	print("Name Already Taken")
	nameID=str(input("Enter Your Name Again: "))
else:
	os.makedirs(path)

while True:
	# Start reading input from webcam, shows webcam video capture on device
	ret,frame=video.read()
	faces=facedetect.detectMultiScale(frame,1.3, 5)
	for x,y,w,h in faces:
		count=count+1
		if not test:
			name='./images/train/'+nameID+'/'+ str(count) + '.jpg'
		else:
			name='./images/val/'+nameID+'/'+ str(count) + '.jpg'
		print("Creating Images........." +name)
		cv2.imwrite(name, frame[y:y+h,x:x+w])
		cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 3)
	cv2.imshow("WindowFrame", frame)
	cv2.waitKey(1)
	if not test and count>10:
		test = True
		continue
	elif test and count>5:
		break

# Stops reading input from webcam, removes webcam video capture on device
video.release()
cv2.destroyAllWindows()

Name Already Taken


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


# Data Preprocessing

Since our data is collected by hand, we do not need to check for missing values. Since our data collection method already makes use of the cascade classifier to ensure that a face is in frame and crops the image down to one of a fixed size and of the face only, we do not need to preprocess the data or check for any images that are not relevant.

Since our aim is to perform one shot learning, we do not need to perform any data augmentation.

# Creating a Model

For this project, we will create 2 models, one using transfer learning of MobileNet and the other being a model similar to FaceNEt, and compare them. Then based on the results, we will select the better one for our final product.

First, we define the size of the images for our model.

In [9]:
target_size = (224, 224)

Next, we load the datasets that we will be using for training.

In [10]:
train = tf.keras.utils.image_dataset_from_directory('images/train', labels = 'inferred', image_size=target_size)
val = tf.keras.utils.image_dataset_from_directory('images/val', labels = 'inferred', image_size=target_size)

Found 426 files belonging to 3 classes.


Found 427 files belonging to 3 classes.


## Deep Learning Model: MobileNet Transfer Learning CNN 

In [11]:
mobile_model = MobileNetV2(weights='imagenet', input_shape = target_size + (3,), include_top=False)
flat_layer = Flatten()(mobile_model.output)
dense_layer_1 = Dense(1024, activation='relu')(flat_layer)
normal_layer_1 = BatchNormalization()(dense_layer_1)
dense_layer_2 = Dense(1024, activation='relu')(normal_layer_1)
normal_layer_2 = BatchNormalization()(dense_layer_2)
dense_layer_3 = Dense(1024, activation='relu')(normal_layer_2)
normal_layer_3 = BatchNormalization()(dense_layer_3)
dense_layer_4 = Dense(256, activation='relu')(normal_layer_3)
normal_layer_4 = BatchNormalization()(dense_layer_4)
output_layer = Dense(3, activation='relu')(normal_layer_4)

for layer in mobile_model.layers:
    layer.trainable = False

transfer_mobile_model = Model(inputs = mobile_model.inputs, outputs = output_layer)

In [12]:
transfer_mobile_model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

transferhistory = transfer_mobile_model.fit(train, validation_data=val, epochs=10)

Epoch 1/10


14/14 [==============================] - 6s 311ms/step - loss: 0.1359 - accuracy: 0.9437 - val_loss: 10.4723 - val_accuracy: 0.6464
Epoch 2/10
14/14 [==============================] - 4s 279ms/step - loss: 0.0329 - accuracy: 0.9930 - val_loss: 9.2059 - val_accuracy: 0.6300
Epoch 3/10
14/14 [==============================] - 4s 281ms/step - loss: 0.0300 - accuracy: 0.9906 - val_loss: 11.1579 - val_accuracy: 0.6230
Epoch 4/10
14/14 [==============================] - 4s 276ms/step - loss: 0.0501 - accuracy: 0.9930 - val_loss: 8.5597 - val_accuracy: 0.6393
Epoch 5/10
14/14 [==============================] - 4s 274ms/step - loss: 0.0082 - accuracy: 0.9977 - val_loss: 4.7214 - val_accuracy: 0.6464
Epoch 6/10
14/14 [==============================] - 4s 277ms/step - loss: 0.0311 - accuracy: 0.9883 - val_loss: 3.9302 - val_accuracy: 0.6862
Epoch 7/10
14/14 [==============================] - 4s 270ms/step - loss: 0.0172 - accuracy: 0.9977 - val_loss: 5.7700 - val_accuracy: 0.7026
Ep

## Deep Learning Model: FaceNet Copy

In [11]:
def inception_block_1a(X):
    X_3=Conv2D(96,(1,1),data_format='channels_first',name='inception_3a_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3a_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(128,(3,3),data_format='channels_first',name='inception_3a_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3a_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    X_5=Conv2D(16,(1,1),data_format='channels_first',name='inception_3a_5x5_conv1')(X)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3a_5x5_bn1')(X_5)
    X_5=Activation('relu')(X_5)
    X_5=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_5)
    X_5=Conv2D(32,(5,5),data_format='channels_first',name='inception_3a_5x5_conv2')(X_5)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3a_5x5_bn2')(X_5)
    X_5=Activation('relu')(X_5)
    
    X_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(X)
    X_pool = Conv2D(32, (1, 1), data_format='channels_first', name='inception_3a_pool_conv')(X_pool)
    X_pool = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3a_pool_bn')(X_pool)
    X_pool = Activation('relu')(X_pool)
    X_pool = ZeroPadding2D(padding=((3, 4), (3, 4)), data_format='channels_first')(X_pool)
    
    X_1=Conv2D(64,(1,1),data_format='channels_first',name='inception_3a_1x1_conv')(X)
    X_1=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3a_1x1_bn')(X_1)
    X_1=Activation('relu')(X_1)
    
    inception=concatenate([X_3,X_5,X_pool,X_1],axis=1)
    return inception

def inception_block_1b(X):
    X_3=Conv2D(96,(1,1),data_format='channels_first',name='inception_3b_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(128,(3,3),data_format='channels_first',name='inception_3b_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    X_5=Conv2D(32,(1,1),data_format='channels_first',name='inception_3b_5x5_conv1')(X)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_5x5_bn1')(X_5)
    X_5=Activation('relu')(X_5)
    X_5=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_5)
    X_5=Conv2D(64,(5,5),data_format='channels_first',name='inception_3b_5x5_conv2')(X_5)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_5x5_bn2')(X_5)
    X_5=Activation('relu')(X_5)
    
    X_P=AveragePooling2D(pool_size=(3,3),strides=(3,3),data_format='channels_first')(X)
    X_P=Conv2D(64,(1,1),data_format='channels_first',name='inception_3b_pool_conv')(X_P)
    X_P=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_pool_bn')(X_P)
    X_P=Activation('relu')(X_P)
    X_P=ZeroPadding2D(padding=(4,4),data_format='channels_first')(X_P)
    
    X_1=Conv2D(64,(1,1),data_format='channels_first',name='inception_3b_1x1_conv')(X)
    X_1=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3b_1x1_bn')(X_1)
    X_1=Activation('relu')(X_1)
    
    inception=concatenate([X_3,X_5,X_P,X_1],axis=1)
    return inception

def inception_block_1c(X):
    X_3=Conv2D(128,(1,1),data_format='channels_first',name='inception_3c_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3c_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(256,(3,3),strides=(2,2),data_format='channels_first',name='inception_3c_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3c_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    X_5=Conv2D(32,(1,1),data_format='channels_first',name='inception_3c_5x5_conv1')(X)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3c_5x5_bn1')(X_5)
    X_5=Activation('relu')(X_5)
    X_5=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_5)
    X_5=Conv2D(64,(5,5),strides=(2,2),data_format='channels_first',name='inception_3c_5x5_conv2')(X_5)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_3c_5x5_bn2')(X_5)
    X_5=Activation('relu')(X_5)
    
    X_P=MaxPooling2D(pool_size=3,strides=2,data_format='channels_first')(X)
    X_P=ZeroPadding2D(padding=((0,1),(0,1)),data_format='channels_first')(X_P)
    

    inception=concatenate([X_3,X_5,X_P],axis=1)
    return inception

def inception_block_2a(X):
    X_3=Conv2D(96,(1,1),data_format='channels_first',name='inception_4a_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(192,(3,3),data_format='channels_first',name='inception_4a_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    X_5=Conv2D(32,(1,1),data_format='channels_first',name='inception_4a_5x5_conv1')(X)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_5x5_bn1')(X_5)
    X_5=Activation('relu')(X_5)
    X_5=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_5)
    X_5=Conv2D(64,(5,5),data_format='channels_first',name='inception_4a_5x5_conv2')(X_5)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_5x5_bn2')(X_5)
    X_5=Activation('relu')(X_5)
    
    X_P=AveragePooling2D(pool_size=(3,3),strides=(3,3),data_format='channels_first')(X)
    X_P=Conv2D(128,(1,1),data_format='channels_first',name='inception_4a_pool_conv')(X_P)
    X_P=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_pool_bn')(X_P)
    X_P=Activation('relu')(X_P)
    X_P=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_P)
    
    X_1=Conv2D(256,(1,1),data_format='channels_first',name='inception_4a_1x1_conv')(X)
    X_1=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4a_1x1_bn')(X_1)
    X_1=Activation('relu')(X_1)
    
    inception=concatenate([X_3,X_5,X_P,X_1],axis=1)
    return inception

def inception_block_2b(X):
    X_3=Conv2D(160,(1,1),data_format='channels_first',name='inception_4e_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4e_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(256,(3,3),strides=(2,2),data_format='channels_first',name='inception_4e_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4e_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    X_5=Conv2D(64,(1,1),data_format='channels_first',name='inception_4e_5x5_conv1')(X)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4e_5x5_bn1')(X_5)
    X_5=Activation('relu')(X_5)
    X_5=ZeroPadding2D(padding=(2,2),data_format='channels_first')(X_5)
    X_5=Conv2D(128,(5,5),strides=(2,2),data_format='channels_first',name='inception_4e_5x5_conv2')(X_5)
    X_5=BatchNormalization(axis=1,epsilon=0.00001,name='inception_4e_5x5_bn2')(X_5)
    X_5=Activation('relu')(X_5)
    
    X_P=MaxPooling2D(pool_size=3,strides=2,data_format='channels_first')(X)
    X_P=ZeroPadding2D(padding=((0,1),(0,1)),data_format='channels_first')(X_P)

    inception=concatenate([X_3,X_5,X_P],axis=1)
    return inception

def inception_block_3a(X):
    X_3=Conv2D(96,(1,1),data_format='channels_first',name='inception_5a_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5a_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(384,(3,3),data_format='channels_first',name='inception_5a_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5a_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)

    
    X_P=AveragePooling2D(pool_size=(3,3),strides=(3,3),data_format='channels_first')(X)
    X_P=Conv2D(96,(1,1),data_format='channels_first',name='inception_5a_pool_conv')(X_P)
    X_P=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5a_pool_bn')(X_P)
    X_P=Activation('relu')(X_P)
    X_P=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_P)
    
    X_1=Conv2D(256,(1,1),data_format='channels_first',name='inception_5a_1x1_conv')(X)
    X_1=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5a_1x1_bn')(X_1)
    X_1=Activation('relu')(X_1)
    
    inception=concatenate([X_3,X_P,X_1],axis=1)
    return inception
def inception_block_3b(X):
    X_3=Conv2D(96,(1,1),data_format='channels_first',name='inception_5b_3x3_conv1')(X)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5b_3x3_bn1')(X_3)
    X_3=Activation('relu')(X_3)
    X_3=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_3)
    X_3=Conv2D(384,(3,3),data_format='channels_first',name='inception_5b_3x3_conv2')(X_3)
    X_3=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5b_3x3_bn2')(X_3)
    X_3=Activation('relu')(X_3)
    
    
    X_P=MaxPooling2D(pool_size=(3,3),strides=2,data_format='channels_first')(X)
    X_P=Conv2D(96,(1,1),data_format='channels_first',name='inception_5b_pool_conv')(X_P)
    X_P=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5b_pool_bn')(X_P)
    X_P=Activation('relu')(X_P)
    X_P=ZeroPadding2D(padding=(1,1),data_format='channels_first')(X_P)
    
    X_1=Conv2D(256,(1,1),data_format='channels_first',name='inception_5b_1x1_conv')(X)
    X_1=BatchNormalization(axis=1,epsilon=0.00001,name='inception_5b_1x1_bn')(X_1)
    X_1=Activation('relu')(X_1)
    
    inception=concatenate([X_3,X_P,X_1],axis=1)
    return inception


def FinalModel(input_shape):
    
    X_input=Input(input_shape)
    
    X=ZeroPadding2D(padding=(3,3))(X_input)
    X=Conv2D(64,(7,7),strides=(2,2),name='conv1')(X)
    X=BatchNormalization(axis=1,name='bn1')(X)
    X=Activation('relu')(X)
    
    X=ZeroPadding2D((1,1))(X)
    X=MaxPooling2D((3,3),strides=2)(X)
    
    X=Conv2D(64,(1,1),strides=(1,1),name='conv2')(X)
    X=BatchNormalization(axis=1,epsilon=0.00001,name='bn2')(X)
    X=Activation('relu')(X) 
    
    X=ZeroPadding2D(padding=(1,1))(X)
    
    X=Conv2D(192,(3,3),strides=(1,1),name='conv3')(X)
    X=BatchNormalization(axis=1,epsilon=0.00001,name='bn3')(X)
    X=Activation('relu')(X)
    
    X=ZeroPadding2D(padding=(1,1))(X)
    X=MaxPooling2D(pool_size=(3,3),strides=(2,2))(X)
    
    X=inception_block_1a(X)
    X=inception_block_1b(X)
    X=inception_block_1c(X)
    
    X=inception_block_2a(X)
    X=inception_block_2b(X)
    
    X=inception_block_3a(X)
    X=inception_block_3b(X)
    
    X=AveragePooling2D(pool_size=(3,3),strides=(1,1),data_format='channels_first')(X)
    X=Flatten()(X)
    X=Dense(128,activation='relu',kernel_initializer='glorot_normal',name='dense_layer')(X)
    X=Lambda(lambda x:K.l2_normalize(x,axis=1),name='lambda_1')(X)
    
    model=Model(inputs=X_input,outputs=X,name='FaceRecognotionModel')
    return model    

# Evaluation of Models

In [37]:
labels = next(os.walk("images/train"))[1]

In [ ]:
count = 0
while True:
	success, imgOrignal=cap.read()
	faces = facedetect.detectMultiScale(imgOrignal,1.3,5)
	for x,y,w,h in faces:
		crop_img=imgOrignal[y:y+h,x:x+h]
		img=cv2.resize(crop_img, (224,224))
		img=img.reshape(1, 224, 224, 3)
		prediction=model.predict(img)
		classIndex = np.argmax(prediction)

		cv2.rectangle(imgOrignal,(x,y),(x+w,y+h),(0,255,0),2)
		cv2.rectangle(imgOrignal, (x,y-40),(x+w, y), (0,255,0),-2)
		cv2.putText(imgOrignal, labels[classIndex],(x,y-10), cv2.FONT_HERSHEY_COMPLEX, 0.75, (255,255,255),1, cv2.LINE_AA)
		count += 1
		if count > 20:
			count = 0
			print('index 0')
			import sys
			sys.exit()
			

	cv2.imshow("Result",imgOrignal)
	k=cv2.waitKey(1)
	if k==ord('q'):
		break


cap.release()
cv2.destroyAllWindows()

# Hyperparameter Tuning

Now, we will tune our model. First, we remake the model that we chose, that being the V3model so that we can tune it.

In [34]:
inceptionthemovie = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False
)
inceptionthemovie.trainable = False
input = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
temp = data_augmentation(input)
temp = tf.keras.applications.inception_v3.preprocess_input(temp)
temp = inceptionthemovie(temp, training=False)
temp = tf.keras.layers.GlobalAveragePooling2D()(temp)

finalmodel = tf.keras.Model(input, tf.keras.layers.Dense(10)(temp))

Then we will use the keras tuner in order to find the best hyperparameters for our model. In this case, only the learning rate is applicable.

In [35]:
import keras_tuner as kt
def model_builder(hp):
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    # hp_dropout = hp.Float('dropout', min_value=0.1, max_value=0.8, step=0.1) #unused
    # hp_dense_units = hp.Int('units', min_value=128, max_value=1024, step=64) #unused

    model = finalmodel

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    
    return model


tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=30,
                     factor=3,
                     directory='dir')

In [36]:
tuner.search(train, epochs=30, validation_data=val)
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 3 Complete [00h 02m 09s]
val_accuracy: 0.8529999852180481

Best val_accuracy So Far: 0.8585000038146973
Total elapsed time: 00h 06m 31s


Finally, we attempt to fit the model to find the epoch with the best accuracy before we do a final evaluation of our model.

In [37]:
tuningmodel = tuner.hypermodel.build(best_hps)
history = tuningmodel.fit(train, epochs=30, validation_data=val, callbacks=[callbacks])

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 590ms/step - accuracy: 0.8345 - loss: 0.5455 - val_accuracy: 0.8660 - val_loss: 0.4584
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 569ms/step - accuracy: 0.8300 - loss: 0.5451 - val_accuracy: 0.8685 - val_loss: 0.4545
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 563ms/step - accuracy: 0.8315 - loss: 0.5270 - val_accuracy: 0.8695 - val_loss: 0.4530
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 565ms/step - accuracy: 0.8352 - loss: 0.5213 - val_accuracy: 0.8705 - val_loss: 0.4512
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 560ms/step - accuracy: 0.8350 - loss: 0.5142 - val_accuracy: 0.8700 - val_loss: 0.4503
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 562ms/step - accuracy: 0.8313 - loss: 0.5212 - val_accuracy: 0.8720 - val_loss: 0.4491
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 565ms/step - accuracy: 0.8308 - loss: 0.5099 - val_accuracy: 0.8695 - val_loss: 0.4486
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 565ms/step - accuracy: 0.8298 - loss: 0

In [38]:
finalmodel = tuner.hypermodel.build(best_hps)

finalmodel.fit(train, epochs=best_epoch, validation_data=val)
finalmodel.evaluate(val)

Epoch 1/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 595ms/step - accuracy: 0.8415 - loss: 0.4734 - val_accuracy: 0.8720 - val_loss: 0.4397
Epoch 2/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 567ms/step - accuracy: 0.8336 - loss: 0.4774 - val_accuracy: 0.8740 - val_loss: 0.4382
Epoch 3/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 564ms/step - accuracy: 0.8455 - loss: 0.4807 - val_accuracy: 0.8760 - val_loss: 0.4379
Epoch 4/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 565ms/step - accuracy: 0.8456 - loss: 0.4779 - val_accuracy: 0.8740 - val_loss: 0.4389
Epoch 5/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 60s 568ms/step - accuracy: 0.8446 - loss: 0.4756 - val_accuracy: 0.8725 - val_loss: 0.4386
Epoch 6/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 567ms/step - accuracy: 0.8478 - loss: 0.4612 - val_accuracy: 0.8750 - val_loss: 0.4384
Epoch 7/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 566ms/step - accuracy: 0.8451 - loss: 0.4666 - val_accuracy: 0.8730 - val_loss: 0.4387
Epoch 8/18
100/100 ━━━━━━━━━━━━━━━━━━━━ 59s 567ms/step - accuracy: 0.8467 - loss: 0

[0.4361916482448578, 0.8755000233650208]

# Final Model Evaluation
```py
before = [0.4680415689945221, 0.8709999918937683]
after = [0.4361916482448578, 0.8755000233650208]
```
From this, we can conclude that finetuning has increased the accuracy slightly by around 0.55% and decreased the loss by around 0.03, which means the hyperparameter tuning has worked and improved the model. Overall, the model is quite accurate and categorising pictures into the 10 different classes.